# データサイエンス100本ノック（構造化データ加工編） - Julia

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- 利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
# ENV["COLUMNS"]=240  # 描画する表の列数を増やす
# ENV["LINES"]=10  # 行の数は制限（問題の指示とは異なるので好みに合わせて修正）
# ENV["PYTHON"]="" # 使用する python へのパス

using Pkg

# Pkg.add("DataFrames")
# Pkg.add("DataFramesMeta")
# Pkg.add("LibPQ")
# Pkg.add("StatsBase")
# Pkg.add("ScikitLearn")
# Pkg.add("MLDataPattern")
# Pkg.add("CSV")
# Pkg.add("ShiftedArrays")

In [2]:
using DataFrames
using DataFramesMeta
using StatsBase
using Statistics
using Dates
using Random
using ScikitLearn
using MLDataPattern
using CSV
using ShiftedArrays: lag

# @sk_import preprocessing: (LabelBinarizer, StandardScaler, MinMaxScaler)

## データ読み込み

In [3]:
if "PG_PORT" in keys(ENV)
    using LibPQ
    
    host = "db"
    port = ENV["PG_PORT"]
    database = ENV["PG_DATABASE"]
    user = ENV["PG_USER"]
    password = ENV["PG_PASSWORD"]
    dsl = "postgresql://$user:$password@$host:$port/$database"
    conn = LibPQ.Connection(dsl)

    # データ読み込み
    df_customer = DataFrame(execute(conn, "select * from customer"))
    df_category = DataFrame(execute(conn, "select * from category"))
    df_product = DataFrame(execute(conn, "select * from product"))
    df_receipt = DataFrame(execute(conn, "select * from receipt"))
    df_store = DataFrame(execute(conn, "select * from store"))
    df_geocode = DataFrame(execute(conn, "select * from geocode"))

    println("load data from DB.")
else
    df_customer = CSV.read("./data/customer.csv", DataFrame)
    df_category = CSV.read("./data/category.csv", DataFrame)
    df_product = CSV.read("./data/product.csv", DataFrame)
    df_receipt = CSV.read("./data/receipt.csv", DataFrame)
    df_store = CSV.read("./data/store.csv", DataFrame)
    df_geocode = CSV.read("./data/geocode.csv", DataFrame)
    
    println("load data from csv.")
end

load data from DB.


# 演習問題

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [4]:
@linq df_receipt |> first(10)

Row,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
,Int64,Int64,String7,Int64,Int64,String15,String15,Int64,Int64
1,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
2,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
3,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
4,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
5,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
6,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
7,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
8,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
9,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770


---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [5]:
@linq df_receipt |>
    select(:sales_ymd, :customer_id, :product_cd, :amount) |>
    first(10)

Row,sales_ymd,customer_id,product_cd,amount
,Int64,String15,String15,Int64
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更して抽出すること。

In [6]:
# @linq df_receipt |>
#     select(:sales_ymd, :customer_id, :product_cd, :amount) |>
#     rename(:sales_ymd => :sales_date) |>
#     first(10)

# 別解. select で rename も可能
@chain df_receipt begin
    select(:sales_ymd => :sales_date, :customer_id, :product_cd, :amount)
    first(10)
end

Row,sales_date,customer_id,product_cd,amount
,Int64,String15,String15,Int64
1,20181103,CS006214000001,P070305012,158
2,20181118,CS008415000097,P070701017,81
3,20170712,CS028414000014,P060101005,170
4,20190205,ZZ000000000000,P050301001,25
5,20180821,CS025415000050,P060102007,90
6,20190605,CS003515000195,P050102002,138
7,20181205,CS024514000042,P080101005,30
8,20190922,CS040415000178,P070501004,128
9,20170504,ZZ000000000000,P071302010,770


---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [7]:
@linq df_receipt |>
    select(:sales_ymd, :customer_id, :product_cd, :amount) |>
    where(:customer_id .== "CS018205000001")

Row,sales_ymd,customer_id,product_cd,amount
,Int64,String15,String15,Int64
1,20180911,CS018205000001,P071401012,2200
2,20180414,CS018205000001,P060104007,600
3,20170614,CS018205000001,P050206001,990
4,20170614,CS018205000001,P060702015,108
5,20190216,CS018205000001,P071005024,102
6,20180414,CS018205000001,P071101002,278
7,20190226,CS018205000001,P070902035,168
8,20190924,CS018205000001,P060805001,495
9,20190226,CS018205000001,P071401020,2200


---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [8]:
@linq df_receipt |>
    select(:sales_ymd, :customer_id, :product_cd, :amount) |>
    where(:customer_id .== "CS018205000001", :amount .>= 1_000)

Row,sales_ymd,customer_id,product_cd,amount
,Int64,String15,String15,Int64
1,20180911,CS018205000001,P071401012,2200
2,20190226,CS018205000001,P071401020,2200
3,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [9]:
@linq df_receipt |>
    select(:sales_ymd, :customer_id, :product_cd, :quantity, :amount) |>
    where(:customer_id .== "CS018205000001", (:amount .>= 1_000).|(:quantity .>= 5))

Row,sales_ymd,customer_id,product_cd,quantity,amount
,Int64,String15,String15,Int64,Int64
1,20180911,CS018205000001,P071401012,1,2200
2,20180414,CS018205000001,P060104007,6,600
3,20170614,CS018205000001,P050206001,5,990
4,20190226,CS018205000001,P071401020,1,2200
5,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [10]:
@linq df_receipt |>
    select(:sales_ymd, :customer_id, :product_cd, :amount) |>
    where(:customer_id .== "CS018205000001", 1_000 .<= :amount .<= 2_000)

Row,sales_ymd,customer_id,product_cd,amount
,Int64,String15,String15,Int64
1,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [11]:
@linq df_receipt |>
    select(:sales_ymd, :customer_id, :product_cd, :amount) |>
    where(:customer_id .== "CS018205000001", :product_cd .!= "P071401019")

Row,sales_ymd,customer_id,product_cd,amount
,Int64,String15,String15,Int64
1,20180911,CS018205000001,P071401012,2200
2,20180414,CS018205000001,P060104007,600
3,20170614,CS018205000001,P050206001,990
4,20170614,CS018205000001,P060702015,108
5,20190216,CS018205000001,P071005024,102
6,20180414,CS018205000001,P071101002,278
7,20190226,CS018205000001,P070902035,168
8,20190924,CS018205000001,P060805001,495
9,20190226,CS018205000001,P071401020,2200


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
> 
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [12]:
@linq df_store |>
    where(:prefecture_cd .!= "13", :floor_area .<= 900)

Row,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String7,String31,Int64,String15,String,String,String15,Float64,Float64,Float64
1,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831.0
2,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.632,35.5466,890.0
3,S13008,成城店,13,東京都,東京都世田谷区成城三丁目,トウキョウトセタガヤクセイジョウサンチョウメ,03-0123-4012,139.597,35.6361,883.0
4,S13020,十条仲原店,13,東京都,東京都北区十条仲原三丁目,トウキョウトキタクジュウジョウナカハラサンチョウメ,03-0123-4019,139.719,35.7669,801.0
5,S13043,南品川店,13,東京都,東京都品川区南品川三丁目,トウキョウトシナガワクミナミシナガワサンチョウメ,03-0123-4027,139.744,35.6098,845.0
6,S13041,八王子店,13,東京都,東京都八王子市大塚,トウキョウトハチオウジシオオツカ,042-123-4026,139.423,35.6379,810.0
7,S13017,高円寺南店,13,東京都,東京都杉並区高円寺南四丁目,トウキョウトスギナミクコウエンジミナミヨンチョウメ,03-0123-4016,139.651,35.7027,841.0
8,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.022,35.6612,808.0
9,S13037,南砂一丁目店,13,東京都,東京都江東区南砂一丁目,トウキョウトコウトウクミナミスナイッチョウメ,03-0123-4023,139.821,35.679,814.0


---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [13]:
@linq df_store |>
    where(startswith.(:store_cd, "S14")) |>
    first(10)

Row,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String7,String31,Int64,String15,String,String,String15,Float64,Float64,Float64
1,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732.0
2,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495.0
3,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.372,35.5733,1679.0
4,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.524,1548.0
5,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830.0
6,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.463,1574.0
7,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.658,35.4216,1341.0
8,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831.0
9,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.579,35.2964,1838.0


---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [14]:
@linq df_customer |>
    where(endswith.(:customer_id, "1")) |>
    first(10)

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
3,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
4,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
5,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
6,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
7,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
8,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
9,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0


---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [15]:
@linq df_store |>
    where(occursin.("横浜市", :address))

Row,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String7,String31,Int64,String15,String,String,String15,Float64,Float64,Float64
1,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732.0
2,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495.0
3,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.524,1548.0
4,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830.0
5,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.463,1574.0
6,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.658,35.4216,1341.0
7,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.592,35.5619,831.0
8,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.632,35.5466,890.0
9,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.576,35.5491,1657.0


---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [16]:
@linq df_customer |>
    where(occursin.(r"^[A-F]", :status_cd)) |>
    first(10)

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
2,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
5,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
6,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
7,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
8,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
9,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E


---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [17]:
@linq df_customer |>
    where(occursin.(r".*[1-9]$", :status_cd)) |>
    first(10)

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
2,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
3,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
4,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
5,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
6,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
7,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
8,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
9,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1


---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [18]:
@linq df_customer |>
    where(occursin.(r"^[A-F].*[1-9]$", :status_cd)) |>
    first(10)

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
2,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
3,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
4,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
5,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
6,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
7,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
8,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
9,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1


---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [19]:
@linq df_store |>
    where(occursin.(r"^[0-9]{3}-[0-9]{3}-[0-9]{4}$", :tel_no))

Row,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
,String7,String31,Int64,String15,String,String,String15,Float64,Float64,Float64
1,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.6356,1698.0
2,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.48,35.7057,1735.0
3,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.633,35.5005,1732.0
4,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.496,35.4592,1495.0
5,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.372,35.5733,1679.0
6,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.499,35.524,1548.0
7,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.496,35.4592,1830.0
8,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.438,35.5529,1087.0
9,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.496,35.463,1574.0


---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [20]:
@linq df_customer |>
    sort(:birth_day) |>
    first(10)

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
2,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
3,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
4,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
5,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
6,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
7,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
8,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
9,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [21]:
@linq df_customer |>
    sort(:birth_day, rev=true) |>
    first(10)

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15
1,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
2,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
3,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
4,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
5,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
6,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
7,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
8,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
9,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F


---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [22]:
@chain df_receipt begin
    @transform(:rank = competerank(:amount, rev=true))
    select(:customer_id, :amount, :rank)
    sort(:amount, rev=true)
    first(10)
end

Row,customer_id,amount,rank
,String15,Int64,Int64
1,CS011415000006,10925,1
2,ZZ000000000000,6800,2
3,CS028605000002,5780,3
4,CS015515000034,5480,4
5,ZZ000000000000,5480,4
6,ZZ000000000000,5480,4
7,ZZ000000000000,5440,7
8,CS021515000089,5440,7
9,CS015515000083,5280,9


---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [23]:
@chain df_receipt begin
    sort(:amount, rev=true)
    @transform(:rank = ordinalrank(:amount, rev=true))
    select(:customer_id, :amount, :rank)
    first(10)
end

Row,customer_id,amount,rank
,String15,Int64,Int64
1,CS011415000006,10925,1
2,ZZ000000000000,6800,2
3,CS028605000002,5780,3
4,CS015515000034,5480,4
5,ZZ000000000000,5480,5
6,ZZ000000000000,5480,6
7,ZZ000000000000,5440,7
8,CS021515000089,5440,8
9,CS015515000083,5280,9


---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [24]:
nrow(df_receipt)

104681

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [25]:
length(unique(df_receipt.customer_id))

8307

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [26]:
@chain df_receipt begin
    sort(:store_cd)
    groupby(:store_cd)
    combine(:amount => sum, :quantity => sum)
end

Row,store_cd,amount_sum,quantity_sum
,String7,Int64,Int64
1,S14006,712839,2284
2,S13008,809288,2491
3,S14028,786145,2458
4,S14042,534689,1935
5,S14025,755581,2394
6,S13003,764294,2197
7,S14024,736323,2417
8,S14040,701858,2233
9,S13020,796383,2383


---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [27]:
@chain df_receipt begin
    sort(:customer_id)
    groupby(:customer_id)
    combine(:sales_ymd => maximum)
    first(10)
end

Row,customer_id,sales_ymd_maximum
,String15,Int64
1,CS001113000004,20190308
2,CS001114000005,20190731
3,CS001115000010,20190405
4,CS001205000004,20190625
5,CS001205000006,20190224
6,CS001211000025,20190322
7,CS001212000027,20170127
8,CS001212000031,20180906
9,CS001212000046,20170811


---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [28]:
@chain df_receipt begin
    sort(:customer_id)
    groupby(:customer_id)
    combine(:sales_ymd => minimum)
    first(10)
end

Row,customer_id,sales_ymd_minimum
,String15,Int64
1,CS001113000004,20190308
2,CS001114000005,20180503
3,CS001115000010,20171228
4,CS001205000004,20170914
5,CS001205000006,20180207
6,CS001211000025,20190322
7,CS001212000027,20170127
8,CS001212000031,20180906
9,CS001212000046,20170811


---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [29]:
@chain df_receipt begin
    sort(:customer_id)
    groupby(:customer_id)
    combine(:sales_ymd => maximum, :sales_ymd => minimum)
    @subset(:sales_ymd_minimum .!= :sales_ymd_maximum)
    first(10)
end

Row,customer_id,sales_ymd_maximum,sales_ymd_minimum
,String15,Int64,Int64
1,CS001114000005,20190731,20180503
2,CS001115000010,20190405,20171228
3,CS001205000004,20190625,20170914
4,CS001205000006,20190224,20180207
5,CS001214000009,20190902,20170306
6,CS001214000017,20191006,20180828
7,CS001214000048,20190929,20171109
8,CS001214000052,20190617,20180208
9,CS001215000005,20181021,20170206


---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [30]:
@chain df_receipt begin
    select(:store_cd, :amount)
    groupby(:store_cd)
    combine(:amount => mean)
    sort(:amount_mean, rev=true)
    first(5)
end

Row,store_cd,amount_mean
,String7,Float64
1,S13052,402.867
2,S13015,351.112
3,S13003,350.916
4,S14010,348.791
5,S13001,348.47


---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [31]:
@chain df_receipt begin
    select(:store_cd, :amount)
    groupby(:store_cd)
    combine(:amount => median)
    sort(:amount_median, rev=true)
    first(5)
end

Row,store_cd,amount_median
,String7,Float64
1,S13052,190.0
2,S14010,188.0
3,S14050,185.0
4,S13003,180.0
5,S14040,180.0


---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [32]:
@chain df_receipt begin
    select(:store_cd, :product_cd)
    sort(:store_cd)
    groupby(:store_cd)
    combine(:product_cd => mode)
    first(10)
end

Row,store_cd,product_cd_mode
,String7,String15
1,S14006,P060303001
2,S13008,P060303001
3,S14028,P060303001
4,S14042,P050101001
5,S14025,P060303001
6,S13003,P071401001
7,S14024,P060303001
8,S14040,P060303001
9,S13020,P071401001


---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [33]:
@chain df_receipt begin
    select(:store_cd, :amount)
    groupby(:store_cd)
    combine(:amount => var)
    sort(:amount_var, rev=true)
    first(5)
end

Row,store_cd,amount_var
,String7,Float64
1,S13052,4.41863e5
2,S14011,3.06442e5
3,S14034,2.97068e5
4,S13001,2.95559e5
5,S13015,2.95427e5


---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [34]:
stdpop(x) = std(x, corrected=false)
@chain df_receipt begin
    select(:store_cd, :amount)
    groupby(:store_cd)
    combine(:amount => stdpop, :amount => std)
    sort(:amount_stdpop, rev=true)
    first(5)
end

Row,store_cd,amount_stdpop,amount_std
,String7,Float64,Float64
1,S13052,663.392,664.728
2,S14011,553.457,553.572
3,S14034,544.904,545.04
4,S13001,543.537,543.653
5,S13015,543.41,543.532


---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [35]:
quantile(df_receipt.amount, [0, 0.25, 0.5, 0.75, 1])

5-element Vector{Float64}:
    10.0
   102.0
   170.0
   288.0
 10925.0

---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [36]:
@chain df_receipt begin
    select(:store_cd, :amount)
    sort(:store_cd)
    groupby(:store_cd)
    combine(:amount => mean)
    @subset(:amount_mean .>= 330)
end

Row,store_cd,amount_mean
,String7,Float64
1,S13003,350.916
2,S13020,337.88
3,S14026,332.341
4,S13015,351.112
5,S14045,330.082
6,S13019,330.209
7,S14011,335.718
8,S13004,330.944
9,S14010,348.791


---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [37]:
df_amount_sum_by_customer = @chain df_receipt begin
    select(:customer_id, :amount)
    @subset(.!startswith.(:customer_id, "Z"))
    groupby(:customer_id)
    combine(:amount => sum)
end
mean(df_amount_sum_by_customer.amount_sum)

2547.742234529256

---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [38]:
df_amount_sum_by_customer = @chain df_receipt begin
    select(:customer_id, :amount)
    @subset(.!startswith.(:customer_id, "Z"))
    sort(:customer_id)
    groupby(:customer_id)
    combine(:amount => sum)
end

mean_amount = mean(df_amount_sum_by_customer.amount_sum)

@chain df_amount_sum_by_customer begin
    @subset(:amount_sum .>= mean_amount)
    first(10)
end

Row,customer_id,amount_sum
,String15,Int64
1,CS001115000010,3044
2,CS001205000006,3337
3,CS001214000009,4685
4,CS001214000017,4132
5,CS001214000052,5639
6,CS001215000040,3496
7,CS001304000006,3726
8,CS001305000005,3485
9,CS001305000011,4370


---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [39]:
@chain df_receipt begin
    innerjoin(df_store[:, [:store_cd, :store_name]], on=:store_cd)
    first(10)
end

Row,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
,Int64,Int64,String7,Int64,Int64,String15,String15,Int64,Int64,String31
1,20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
2,20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81,成城店
3,20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170,二ツ橋店
4,20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25,新山下店
5,20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90,大和店
6,20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138,狛江店
7,20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30,三田店
8,20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128,長津田店
9,20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770,十条仲原店


---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [40]:
merge_lst = [:category_major_cd, :category_medium_cd, :category_small_cd]
@chain innerjoin(df_product, select(df_category, cat(merge_lst, [:category_small_name], dims=1)), on=merge_lst) begin
    first(10)
end

Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
,String15,Int64,Int64,Int64,Int64?,Int64?,String
1,P040101001,4,401,40101,198,149,弁当類
2,P040101002,4,401,40101,218,164,弁当類
3,P040101003,4,401,40101,230,173,弁当類
4,P040101004,4,401,40101,248,186,弁当類
5,P040101005,4,401,40101,268,201,弁当類
6,P040101006,4,401,40101,298,224,弁当類
7,P040101007,4,401,40101,338,254,弁当類
8,P040101008,4,401,40101,420,315,弁当類
9,P040101009,4,401,40101,498,374,弁当類


---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [41]:
@chain df_customer begin
    @subset(string.(:gender_cd) .== "1", .!startswith.(:customer_id, "Z"))
    leftjoin(df_receipt, on=:customer_id)
    select(:customer_id, :amount)
    coalesce.(0)
    sort(:customer_id)
    groupby(:customer_id)
    combine(:amount => sum)
    first(10)
end

Row,customer_id,amount_sum
,String15,Int64
1,CS001112000009,0
2,CS001112000019,0
3,CS001112000021,0
4,CS001112000023,0
5,CS001112000024,0
6,CS001112000029,0
7,CS001112000030,0
8,CS001113000004,1298
9,CS001113000010,0


---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [42]:
df_receipt_top_20_seles_days = @chain df_receipt begin
    @subset(.!startswith.(:customer_id, "Z"))
    groupby(:customer_id)
    combine(:sales_ymd => length ∘ unique)
    sort(:sales_ymd_length_unique, rev=true)
    first(20)
end

Row,customer_id,sales_ymd_length_unique
,String15,Int64
1,CS040214000008,23
2,CS015415000185,22
3,CS010214000010,22
4,CS010214000002,21
5,CS028415000007,21
6,CS017415000097,20
7,CS016415000141,20
8,CS039414000052,19
9,CS031414000051,19


In [43]:
df_receipt_top_20_customer = @chain df_receipt begin
    @subset(.!startswith.(:customer_id, "Z"))
    groupby(:customer_id)
    combine(:amount => sum)
    sort(:amount_sum, rev=true)
    first(20)
end

Row,customer_id,amount_sum
,String15,Int64
1,CS017415000097,23086
2,CS015415000185,20153
3,CS031414000051,19202
4,CS028415000007,19127
5,CS001605000009,18925
6,CS010214000010,18585
7,CS016415000141,18372
8,CS006515000023,18372
9,CS011414000106,18338


In [44]:
outerjoin(df_receipt_top_20_seles_days, df_receipt_top_20_customer, on=:customer_id)

Row,customer_id,sales_ymd_length_unique,amount_sum
,String15,Int64?,Int64?
1,CS017415000097,20,23086
2,CS015415000185,22,20153
3,CS031414000051,19,19202
4,CS028415000007,21,19127
5,CS010214000010,22,18585
6,CS016415000141,20,18372
7,CS040214000008,23,missing
8,CS010214000002,21,missing
9,CS039414000052,19,missing


---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [45]:
nrow(df_store) * nrow(df_product)

531590

---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [46]:
df_amount_by_date = @chain df_receipt begin
    groupby(:sales_ymd)
    combine(:amount => sum)
    @transform(:amount_diff = lag(:amount_sum) .- :amount_sum)
    first(10)
end

Row,sales_ymd,amount_sum,amount_diff
,Int64,Int64,Int64?
1,20170101,33723,missing
2,20170102,24165,9558
3,20170103,27503,-3338
4,20170104,36165,-8662
5,20170105,37830,-1665
6,20170106,32387,5443
7,20170107,23415,8972
8,20170108,24737,-1322
9,20170109,26718,-1981


---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [47]:
@chain df_receipt begin
    groupby(:sales_ymd)
    combine(:amount => sum)
    @transform(:sales_ymd_pre1 = lag(:sales_ymd), :amount_pre1 = lag(:amount_sum))
    @transform(:sales_ymd_pre2 = lag(:sales_ymd_pre1), :amount_pre2 = lag(:amount_pre1))
    @transform(:sales_ymd_pre3 = lag(:sales_ymd_pre2), :amount_pre3 = lag(:amount_pre2))
    dropmissing()
    first(10)
end

Row,sales_ymd,amount_sum,sales_ymd_pre1,amount_pre1,sales_ymd_pre2,amount_pre2,sales_ymd_pre3,amount_pre3
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,20170104,36165,20170103,27503,20170102,24165,20170101,33723
2,20170105,37830,20170104,36165,20170103,27503,20170102,24165
3,20170106,32387,20170105,37830,20170104,36165,20170103,27503
4,20170107,23415,20170106,32387,20170105,37830,20170104,36165
5,20170108,24737,20170107,23415,20170106,32387,20170105,37830
6,20170109,26718,20170108,24737,20170107,23415,20170106,32387
7,20170110,20143,20170109,26718,20170108,24737,20170107,23415
8,20170111,24287,20170110,20143,20170109,26718,20170108,24737
9,20170112,23526,20170111,24287,20170110,20143,20170109,26718


---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [48]:
df_sales_summary = @chain df_customer begin
    @transform(:period = string.(:age .÷ 10 .* 10, "代"))
    innerjoin(df_receipt, on=:customer_id)
    select(:gender_cd, :period, :amount)
    groupby([:gender_cd, :period])
    combine(:amount => sum)
    unstack(:gender_cd, :amount_sum)
    sort(:period)
end

Row,period,1,0,9
,String,Int64?,Int64?,Int64?
1,10代,149836,1591,4317
2,20代,1363724,72940,44328
3,30代,693047,177322,50441
4,40代,9320791,19355,483512
5,50代,6685192,54320,342923
6,60代,987741,272469,71418
7,70代,29764,13435,2427
8,80代,262923,46360,5111
9,90代,6260,missing,missing


---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [49]:
function change_gender_cd(x::String)
    if x == "1"
        return "01"
    elseif x == "0"
        return "00"
    end
    return "99"
end

@chain df_sales_summary begin
    stack(names(df_sales_summary)[2:end], variable_name=:gender_cd, value_name=:amount_sum)
    dropmissing()
    @transform(:gender_cd = change_gender_cd.(:gender_cd))
    sort([:period, :gender_cd])
end

Row,period,gender_cd,amount_sum
,String,String,Int64
1,10代,00,1591
2,10代,01,149836
3,10代,99,4317
4,20代,00,72940
5,20代,01,1363724
6,20代,99,44328
7,30代,00,177322
8,30代,01,693047
9,30代,99,50441


---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [50]:
@chain df_customer begin
    @transform(:birthday_ymd = Dates.format.(:birth_day, "YYYYmmdd"))
    select(:customer_id, :birthday_ymd)
    first(10)
end

Row,customer_id,birthday_ymd
,String15,String
1,CS021313000114,19810429
2,CS037613000071,19520401
3,CS031415000172,19761004
4,CS028811000001,19330327
5,CS001215000145,19950329
6,CS020401000016,19740915
7,CS015414000103,19770809
8,CS029403000008,19730817
9,CS015804000004,19310502


---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [51]:
@chain df_customer begin
    @transform(:application_date = Date.(string.(:application_date), "YYYYmmdd"))
    select(:customer_id, :application_date)
    first(10)
end

Row,customer_id,application_date
,String15,Date
1,CS021313000114,2015-09-05
2,CS037613000071,2015-04-14
3,CS031415000172,2015-05-29
4,CS028811000001,2016-01-15
5,CS001215000145,2017-06-05
6,CS020401000016,2015-02-25
7,CS015414000103,2015-07-22
8,CS029403000008,2015-05-15
9,CS015804000004,2015-06-07


---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [52]:
@chain df_receipt begin
    @transform(:sales_ymd = Date.(string.(:sales_ymd), "YYYYmmdd"))
    select(:receipt_no, :receipt_sub_no, :sales_ymd)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_ymd
,Int64,Int64,Date
1,112,1,2018-11-03
2,1132,2,2018-11-18
3,1102,1,2017-07-12
4,1132,1,2019-02-05
5,1102,2,2018-08-21
6,1112,1,2019-06-05
7,1102,2,2018-12-05
8,1102,1,2019-09-22
9,1112,2,2017-05-04


---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [53]:
@chain df_receipt begin
    @transform(:sales_ymd_from_unix = Date.(unix2datetime.(:sales_epoch)))
    select(:receipt_no, :receipt_sub_no, :sales_ymd_from_unix)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_ymd_from_unix
,Int64,Int64,Date
1,112,1,2018-11-03
2,1132,2,2018-11-18
3,1102,1,2017-07-12
4,1132,1,2019-02-05
5,1102,2,2018-08-21
6,1112,1,2019-06-05
7,1102,2,2018-12-05
8,1102,1,2019-09-22
9,1112,2,2017-05-04


---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [54]:
@chain df_receipt begin
    @transform(:sales_year = year.(unix2datetime.(:sales_epoch)))
    select(:receipt_no, :receipt_sub_no, :sales_year)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_year
,Int64,Int64,Int64
1,112,1,2018
2,1132,2,2018
3,1102,1,2017
4,1132,1,2019
5,1102,2,2018
6,1112,1,2019
7,1102,2,2018
8,1102,1,2019
9,1112,2,2017


---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [55]:
@chain df_receipt begin
    @transform(:sales_month = lpad.(string.(month.(unix2datetime.(:sales_epoch))), 2, "0"))
    select(:receipt_no, :receipt_sub_no, :sales_month)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_month
,Int64,Int64,String
1,112,1,11
2,1132,2,11
3,1102,1,07
4,1132,1,02
5,1102,2,08
6,1112,1,06
7,1102,2,12
8,1102,1,09
9,1112,2,05


---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [56]:
@chain df_receipt begin
    @transform(:sales_day = lpad.(string.(day.(unix2datetime.(:sales_epoch))), 2, "0"))
    select(:receipt_no, :receipt_sub_no, :sales_day)
    first(10)
end

Row,receipt_no,receipt_sub_no,sales_day
,Int64,Int64,String
1,112,1,03
2,1132,2,18
3,1102,1,12
4,1132,1,05
5,1102,2,21
6,1112,1,05
7,1102,2,05
8,1102,1,22
9,1112,2,04


---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [57]:
@chain df_receipt begin
    @subset(.!startswith.(:customer_id, "Z"))
    groupby(:customer_id)
    combine(:amount => sum)
    @transform(:amount_is_over_2000 = ifelse.(:amount_sum .> 2000, 1, 0))
    sort(:customer_id)
    first(10)
end

Row,customer_id,amount_sum,amount_is_over_2000
,String15,Int64,Int64
1,CS001113000004,1298,0
2,CS001114000005,626,0
3,CS001115000010,3044,1
4,CS001205000004,1988,0
5,CS001205000006,3337,1
6,CS001211000025,456,0
7,CS001212000027,448,0
8,CS001212000031,296,0
9,CS001212000046,228,0


---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [58]:
function extract_three_from_postal(x::AbstractString)::Int
    m = match(r"([0-9]{3})-[0-9]{4}", x)
    return parse(Int, m.captures[1])
end

df_customer_in_tokyo = @chain df_customer begin
    @transform(:postal_three = extract_three_from_postal.(:postal_cd))
    @transform(:is_in_tokyo = Int.(100 .<= :postal_three .<= 209))
end

Row,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,postal_three,is_in_tokyo
,String15,String31,Int64,String7,Date,Int64,String15,String,String7,Int64,String15,Int64,Int64
1,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,259,0
2,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,136,1
3,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,151,1
4,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,245,0
5,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,144,1
6,CS020401000016,宮下 達士,0,男性,1974-09-15,44,174-0065,東京都板橋区若木**********,S13020,20150225,0-00000000-0,174,1
7,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B,136,1
8,CS029403000008,釈 人志,0,男性,1973-08-17,45,279-0003,千葉県浦安市海楽**********,S12029,20150515,0-00000000-0,279,0
9,CS015804000004,松谷 米蔵,0,男性,1931-05-02,87,136-0073,東京都江東区北砂**********,S13015,20150607,0-00000000-0,136,1


In [59]:
@chain df_customer_in_tokyo begin
    innerjoin(df_receipt, on=:customer_id)
    unique([:is_in_tokyo, :customer_id])
    groupby(:is_in_tokyo)
    combine(nrow => :customer_num)
end

Row,is_in_tokyo,customer_num
,Int64,Int64
1,0,3906
2,1,4400


---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [60]:
function make_address_cd(x::String)::Int
    m = match(r"(.+)[県都]", x)
    prefecture = m.captures[1]

    if prefecture == "埼玉"
        return 11
    elseif prefecture == "千葉"
        return 12
    elseif prefecture == "東京"
        return 13
    else
        return 14
    end
end

@chain df_customer begin
    @transform(:address_cd = make_address_cd.(:address))
    select(:customer_id, :address, :address_cd)
    first(10)
end

Row,customer_id,address,address_cd
,String15,String,Int64
1,CS021313000114,神奈川県伊勢原市粟窪**********,14
2,CS037613000071,東京都江東区南砂**********,13
3,CS031415000172,東京都渋谷区代々木**********,13
4,CS028811000001,神奈川県横浜市泉区和泉町**********,14
5,CS001215000145,東京都大田区仲六郷**********,13
6,CS020401000016,東京都板橋区若木**********,13
7,CS015414000103,東京都江東区北砂**********,13
8,CS029403000008,千葉県浦安市海楽**********,12
9,CS015804000004,東京都江東区北砂**********,13


---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [61]:
df_amount = @chain df_receipt begin
    groupby(:customer_id)
    combine(:amount => sum)
end

quantiles = quantile(df_amount.amount_sum, [1/4, 1/2, 3/4])

3-element Vector{Float64}:
  548.5
 1478.0
 3651.0

In [62]:
function categorize_quantile(x::Int, quantiles::Array)::Int8
    for (idx, q) in enumerate(quantiles)
        x < q && return idx
    end

    return length(quantiles) + 1
end

categorize_quantile(x::Int) = categorize_quantile(x, quantiles)

@chain df_amount begin
    @transform(:amount_category = categorize_quantile.(:amount_sum))
    select(:customer_id, :amount_sum, :amount_category)
    sort(:customer_id)
    first(10)
end

Row,customer_id,amount_sum,amount_category
,String15,Int64,Int8
1,CS001113000004,1298,2
2,CS001114000005,626,2
3,CS001115000010,3044,3
4,CS001205000004,1988,3
5,CS001205000006,3337,3
6,CS001211000025,456,1
7,CS001212000027,448,1
8,CS001212000031,296,1
9,CS001212000046,228,1


---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [63]:
function categorize_age(x::Int)::Int
    if x >= 60
        return 6
    end
    return x ÷ 10
end

df_customer_era = @chain df_customer begin
    @transform(:category_era = categorize_age.(:age))
    select(:customer_id, :birth_day, :category_era, :gender_cd)
    first(10)
end

Row,customer_id,birth_day,category_era,gender_cd
,String15,Date,Int64,Int64
1,CS021313000114,1981-04-29,3,1
2,CS037613000071,1952-04-01,6,9
3,CS031415000172,1976-10-04,4,1
4,CS028811000001,1933-03-27,6,1
5,CS001215000145,1995-03-29,2,1
6,CS020401000016,1974-09-15,4,0
7,CS015414000103,1977-08-09,4,1
8,CS029403000008,1973-08-17,4,0
9,CS015804000004,1931-05-02,6,0


---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [64]:
@chain df_customer_era begin
    @transform(:category_era_gender = string.(:category_era, "_", :gender_cd))
    first(10)
end

Row,customer_id,birth_day,category_era,gender_cd,category_era_gender
,String15,Date,Int64,Int64,String
1,CS021313000114,1981-04-29,3,1,3_1
2,CS037613000071,1952-04-01,6,9,6_9
3,CS031415000172,1976-10-04,4,1,4_1
4,CS028811000001,1933-03-27,6,1,6_1
5,CS001215000145,1995-03-29,2,1,2_1
6,CS020401000016,1974-09-15,4,0,4_0
7,CS015414000103,1977-08-09,4,1,4_1
8,CS029403000008,1973-08-17,4,0,4_0
9,CS015804000004,1931-05-02,6,0,6_0


---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [113]:
lst_gender_cd = unique(df_customer.gender_cd)

df_customer_with_gender_dummy = @chain df_customer begin
    select(:customer_id, :gender_cd)
    sort(:customer_id)
end

for g_c in lst_gender_cd
    df_customer_with_gender_dummy[!, string("gender_cd_dummy_", g_c)] = Int.(df_customer_with_gender_dummy.gender_cd .== g_c)
end

first(df_customer_with_gender_dummy, 10)

Row,customer_id,gender_cd,gender_cd_dummy_1,gender_cd_dummy_9,gender_cd_dummy_0
,String15,Int64,Int64,Int64,Int64
1,CS001105000001,0,0,0,1
2,CS001112000009,1,1,0,0
3,CS001112000019,1,1,0,0
4,CS001112000021,1,1,0,0
5,CS001112000023,1,1,0,0
6,CS001112000024,1,1,0,0
7,CS001112000029,1,1,0,0
8,CS001112000030,1,1,0,0
9,CS001113000004,1,1,0,0


---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [126]:
@chain df_receipt begin
    @subset(.!startswith.(:customer_id, "Z"))
    groupby(:customer_id)
    combine(:amount => sum)
    @transform(:amount_normalized = (:amount_sum .- mean(:amount_sum)) ./ std(:amount_sum))
    # @transform(:amount_normalized = zscore(:amount_sum)) # 別解
    sort(:customer_id)
    first(10)
end

Row,customer_id,amount_sum,amount_normalized
,String15,Int64,Float64
1,CS001113000004,1298,-0.45935
2,CS001114000005,626,-0.706348
3,CS001115000010,3044,0.182403
4,CS001205000004,1988,-0.205737
5,CS001205000006,3337,0.290096
6,CS001211000025,456,-0.768832
7,CS001212000027,448,-0.771773
8,CS001212000031,296,-0.827641
9,CS001212000046,228,-0.852635


---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [127]:
@chain df_receipt begin
    @subset(.!startswith.(:customer_id, "Z"))
    groupby(:customer_id)
    combine(:amount => sum)
    @transform(:amount_spaned = (:amount_sum .- minimum(:amount_sum)) ./ (maximum(:amount_sum) - minimum(:amount_sum)))
    sort(:customer_id)
    first(10)
end

Row,customer_id,amount_sum,amount_spaned
,String15,Int64,Float64
1,CS001113000004,1298,0.0533542
2,CS001114000005,626,0.0241571
3,CS001115000010,3044,0.129214
4,CS001205000004,1988,0.0833333
5,CS001205000006,3337,0.141945
6,CS001211000025,456,0.0167709
7,CS001212000027,448,0.0164234
8,CS001212000031,296,0.00981926
9,CS001212000046,228,0.00686479


---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [128]:
@chain df_receipt begin
    @subset(.!startswith.(:customer_id, "Z"))
    groupby(:customer_id)
    combine(:amount => sum)
    @transform(:amount_sum_log = log10.(:amount_sum))
    sort(:customer_id)
    first(10)
end

Row,customer_id,amount_sum,amount_sum_log
,String15,Int64,Float64
1,CS001113000004,1298,3.11327
2,CS001114000005,626,2.79657
3,CS001115000010,3044,3.48344
4,CS001205000004,1988,3.29842
5,CS001205000006,3337,3.52336
6,CS001211000025,456,2.65896
7,CS001212000027,448,2.65128
8,CS001212000031,296,2.47129
9,CS001212000046,228,2.35793


---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [129]:
@chain df_receipt begin
    @subset(.!startswith.(:customer_id, "Z"))
    groupby(:customer_id)
    combine(:amount => sum)
    @transform(:amount_sum_log = log.(:amount_sum))
    sort(:customer_id)
    first(10)
end

Row,customer_id,amount_sum,amount_sum_log
,String15,Int64,Float64
1,CS001113000004,1298,7.16858
2,CS001114000005,626,6.43935
3,CS001115000010,3044,8.02093
4,CS001205000004,1988,7.59488
5,CS001205000006,3337,8.11283
6,CS001211000025,456,6.12249
7,CS001212000027,448,6.10479
8,CS001212000031,296,5.69036
9,CS001212000046,228,5.42935


---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [4]:
@chain df_product begin
    @transform(:unit_benefit = :unit_price .- :unit_cost)
    first(10)
end

Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_benefit
,String?,String?,String?,String?,Int32?,Int32?,Int32?
1,P040101001,04,0401,040101,198,149,49
2,P040101002,04,0401,040101,218,164,54
3,P040101003,04,0401,040101,230,173,57
4,P040101004,04,0401,040101,248,186,62
5,P040101005,04,0401,040101,268,201,67
6,P040101006,04,0401,040101,298,224,74
7,P040101007,04,0401,040101,338,254,84
8,P040101008,04,0401,040101,420,315,105
9,P040101009,04,0401,040101,498,374,124


---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [5]:
mean(skipmissing((df_product.unit_price .- df_product.unit_cost) ./ df_product.unit_price))

0.24911389885176904

---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [9]:
@chain df_product begin
    @transform(:unit_price_benefit_30 = floor.(:unit_cost * 10 / 7))
    @transform(:unit_benefit = (:unit_price_benefit_30 .- :unit_cost) ./ :unit_price_benefit_30)
    first(10)
end

Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_price_benefit_30,unit_benefit
,String?,String?,String?,String?,Int32?,Int32?,Float64?,Float64?
1,P040101001,04,0401,040101,198,149,212.0,0.29717
2,P040101002,04,0401,040101,218,164,234.0,0.299145
3,P040101003,04,0401,040101,230,173,247.0,0.299595
4,P040101004,04,0401,040101,248,186,265.0,0.298113
5,P040101005,04,0401,040101,268,201,287.0,0.299652
6,P040101006,04,0401,040101,298,224,320.0,0.3
7,P040101007,04,0401,040101,338,254,362.0,0.298343
8,P040101008,04,0401,040101,420,315,450.0,0.3
9,P040101009,04,0401,040101,498,374,534.0,0.299625


---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [11]:
@chain df_product begin
    @transform(:unit_price_benefit_30_rounding = round.(:unit_cost * 10 / 7))
    @transform(:unit_benefit = (:unit_price_benefit_30_rounding .- :unit_cost) ./ :unit_price_benefit_30_rounding)
    first(10)
end

Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_price_benefit_30_rounding,unit_benefit
,String?,String?,String?,String?,Int32?,Int32?,Float64?,Float64?
1,P040101001,04,0401,040101,198,149,213.0,0.300469
2,P040101002,04,0401,040101,218,164,234.0,0.299145
3,P040101003,04,0401,040101,230,173,247.0,0.299595
4,P040101004,04,0401,040101,248,186,266.0,0.300752
5,P040101005,04,0401,040101,268,201,287.0,0.299652
6,P040101006,04,0401,040101,298,224,320.0,0.3
7,P040101007,04,0401,040101,338,254,363.0,0.300275
8,P040101008,04,0401,040101,420,315,450.0,0.3
9,P040101009,04,0401,040101,498,374,534.0,0.299625


---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [12]:
@chain df_product begin
    @transform(:unit_price_benefit_30_ceil = ceil.(:unit_cost * 10 / 7))
    @transform(:unit_benefit = (:unit_price_benefit_30_ceil .- :unit_cost) ./ :unit_price_benefit_30_ceil)
    first(10)
end

Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_price_benefit_30_ceil,unit_benefit
,String?,String?,String?,String?,Int32?,Int32?,Float64?,Float64?
1,P040101001,04,0401,040101,198,149,213.0,0.300469
2,P040101002,04,0401,040101,218,164,235.0,0.302128
3,P040101003,04,0401,040101,230,173,248.0,0.302419
4,P040101004,04,0401,040101,248,186,266.0,0.300752
5,P040101005,04,0401,040101,268,201,288.0,0.302083
6,P040101006,04,0401,040101,298,224,320.0,0.3
7,P040101007,04,0401,040101,338,254,363.0,0.300275
8,P040101008,04,0401,040101,420,315,450.0,0.3
9,P040101009,04,0401,040101,498,374,535.0,0.300935


---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [14]:
@chain df_product begin
    @transform(:unit_price_with_tax = floor.(:unit_price * 1.1))
    first(10)
end

Row,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,unit_price_with_tax
,String?,String?,String?,String?,Int32?,Int32?,Float64?
1,P040101001,04,0401,040101,198,149,217.0
2,P040101002,04,0401,040101,218,164,239.0
3,P040101003,04,0401,040101,230,173,253.0
4,P040101004,04,0401,040101,248,186,272.0
5,P040101005,04,0401,040101,268,201,294.0
6,P040101006,04,0401,040101,298,224,327.0
7,P040101007,04,0401,040101,338,254,371.0
8,P040101008,04,0401,040101,420,315,462.0
9,P040101009,04,0401,040101,498,374,547.0


---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

---
> P-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

---
> P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

---
> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

---
> P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

---
> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

---
> P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

---
> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

---
> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

---
> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

---
> P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

---
> P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

# これで１００本終わりです。おつかれさまでした！